In [7]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path

from db_psql_model import DatabaseCursor
from utils import get_season, nfl_weeks_pull, game_keys_pull
from yahoo_query import league_season_data

# TODAY = np.datetime64("today", "D")
# TODAY = np.datetime64("2021-09-28")
# TODAY = np.datetime64("2020-09-28")
# TODAY = np.datetime64("2019-09-28")
# TODAY = np.datetime64("2018-09-28")
TODAY = np.datetime64("2017-09-28")
# TODAY = np.datetime64("2016-09-28")
# TODAY = np.datetime64("2015-09-28")
# TODAY = np.datetime64("2014-09-28")
# TODAY = np.datetime64("2013-09-28")
# TODAY = np.datetime64("2012-09-28")

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
NFL_WEEKS = nfl_weeks_pull()
GAME_KEYS = game_keys_pull(first="no")
# SEASON = get_season()
SEASON = TODAY.astype("datetime64[Y]").item().year
LEAGUE_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["league_ID"].values[0]
GAME_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["game_id"].values[0]
nfl_weeks_list = list(NFL_WEEKS["week"][NFL_WEEKS["game_id"] == GAME_ID])

try:
    with open(PATH) as file:
        credentials = yaml.load(file, Loader=yaml.FullLoader)

except Exception as error:
    print(error)

CONSUMER_KEY = credentials["YFPY_CONSUMER_KEY"]
CONSUMER_SECRET = credentials["YFPY_CONSUMER_SECRET"]

league = league_season_data(
    auth_dir=PATH.parent,
    league_id=LEAGUE_ID,
    game_id=GAME_ID,
    game_code="nfl",
    offline=False,
    all_output_as_json=False,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    browser_callback=True,
)

teams = league.teams_and_standings(first_time="no")
teams

[2022-07-07 00:57:04,193 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7ff47776c7c0; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=prod'', closed: 0>
[2022-07-07 00:57:04,195 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
[2022-07-07 00:57:04,195 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-07 00:57:04,196 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
[2022-07-07 00:57:04,196 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14.4 (Ubuntu 14.4-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0, 64-bit',)
[2022-07-07 00:57:04,197 INFO] [sqlalchemy.engine.Engine.base._execute_context] select current_schema()
[2022-07-07 00:57:04,197 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-07 00:57:04,198 DEBUG] [sqlalche

Successfully pulled: SELECT * FROM prod.nflweeks
Successfully pulled: SELECT * FROM prod.gamekeys


[2022-07-07 00:57:05,050 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7ff47776d1c0; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
[2022-07-07 00:57:05,051 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
[2022-07-07 00:57:05,052 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-07 00:57:05,053 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
[2022-07-07 00:57:05,053 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14.4 (Ubuntu 14.4-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0, 64-bit',)
[2022-07-07 00:57:05,054 INFO] [sqlalchemy.engine.Engine.base._execute_context] select current_schema()
[2022-07-07 00:57:05,054 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-07 00:57:05,055 DEBUG] [sqlalchem

Error syntax error at or near "Composed"
LINE 1: COPY (Composed([SQL('SELECT * FROM dev.leagueteams WHERE gam...
              ^

 Query unsuccessful.
Error invalid input syntax for type bigint: "Brady got' Dak"
CONTEXT:  COPY leagueteams, line 1, column has_draft_grade: "Brady got' Dak"

Upload unsuccessful: leagueteams


,clinched_playoffs,draft_grade,has_draft_grade,name,number_of_moves,number_of_trades,team_id,team_key,waiver_priority,roster_adds.coverage_value,...,team_standings.playoff_seed,team_standings.points_against,team_standings.points_for,team_standings.rank,team_standings.streak.type,team_standings.streak.value,manager_id,game_id,league_id,nickname
0,1.0,B,1,Brady got' Dak,13,0,9,371.l.525547.t.9,9,17,...,1,1268.30,1483.68,1,win,7,9,371,525547,Chris
1,1.0,C,1,Krispy Kareem,56,3,4,371.l.525547.t.4,10,17,...,6,1409.92,1511.74,2,loss,5,4,371,525547,Greg
2,1.0,C,1,Give Goodell Zeke-a,29,2,8,371.l.525547.t.8,4,17,...,2,1379.74,1507.94,3,loss,1,8,371,525547,Wes
3,1.0,A-,1,99 probs DEF aint 1,41,0,3,371.l.525547.t.3,6,17,...,4,1285.58,1305.82,4,win,1,3,371,525547,Ryan
4,1.0,B,1,O.J.'s Parole Board,41,2,1,371.l.525547.t.1,8,17,...,5,1448.22,1479.72,5,win,2,1,371,525547,Tim
5,1.0,A,1,Earth to JBear!,91,7,2,371.l.525547.t.2,7,17,...,3,1230.06,1402.30,6,win,2,2,371,525547,James
6,NaN,B+,1,The Crow Show,27,1,5,371.l.525547.t.5,5,17,...,8,1319.94,1061.12,7,loss,4,5,371,525547,Pete
7,NaN,B+,1,Push Ups in Prison,19,1,7,371.l.525547.t.7,2,17,...,7,1371.10,1303.58,8,win,2,7,371,525547,Pat
8,NaN,B,1,Vinatierrible,0,0,10,371.l.525547.t.10,1,17,...,10,1264.80,1090.12,9,loss,2,10,371,525547,Jeremy
9,NaN,C,1,Deckers Barn animals,29,2,6,371.l.525547.t.6,3,17,...,9,1455.82,1287.46,10,loss,2,6,371,525547,Kevin


In [1]:
# league = league_season_data(
#     auth_dir=PATH.parent,
#     league_id=LEAGUE_ID,
#     game_id=GAME_ID,
#     game_code="nfl",
#     offline=False,
#     all_output_as_json=False,
#     consumer_key=CONSUMER_KEY,
#     consumer_secret=CONSUMER_SECRET,
#     browser_callback=True,
# )
# game_keys = league.all_game_keys()
# nfl_weeks = league.all_nfl_weeks()

# meta = league.metadata(first_time=first_time)
# settings, roster, stat_cat = league.set_roster_pos_stat_cat(first_time=first_time)
# draft = league.draft_results(first_time=first_time)
# teams = league.teams_and_standings(first_time="no")

# for week in nfl_weeks_list[:-1]:
#     matchups = league.matchups_by_week(first_time='no', nfl_week=week)

# players = league.players_list(first_time="no")
# players.to_csv(f"players_{str(SEASON)}.csv", index=False)

# for week in nfl_weeks_list[:-1]:
#     week_roster = league.team_roster_by_week(first_time='no', nfl_week=week)
# week_stats = league.player_stats_by_week(first_time="yes", nfl_week=1)
# for week in nfl_weeks_list[1:-1]:
#     week_stats = league.player_stats_by_week(first_time="no", nfl_week=week)

# pd.set_option("display.max_columns", 999)
# week_stats

In [ ]:
# cur = DatabaseCursor(PATH / "private.yaml", options="-c search_path=dev")
# df = pd.read_csv(PATH / "assests" / "game_keys.csv")
# cur.copy_table_to_postgres_new(df, "test", "no")

In [ ]:
# sql_query = """
# SELECT *
# FROM dev.test
# """
# db_cursor = DatabaseCursor(PATH / "private.yaml", options="-c search_path=dev")
# db_cursor.copy_data_from_postgres(sql_query)